# captum for HEK293

## Imports and functions

In [1]:
from IPython.display import clear_output

In [2]:
from google.colab import drive
drive.mount('/content/drive')
clear_output()

In [3]:
!pip install captum
!pip install transformers
!pip install pybedtools
!pip install fuc

clear_output()

In [ ]:
import numpy as np
import pandas as pd
import pybedtools
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer
import torch
import matplotlib.pyplot as plt

In [ ]:
from transformers import AutoTokenizer, BigBirdForSequenceClassification, AutoModelForSequenceClassification, AutoModel

tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/SE_annotation_project/HEK293/results_bert_2/bigbird_base')
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/SE_annotation_project/HEK293/results_bert_2/bigbird_base')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
model.zero_grad()

In [ ]:
def predict(inputs):
    return model(inputs)[0]

def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):

    text_ids = tokenizer.encode(text, add_special_tokens=False)
    # construct input token ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    # construct reference token ids
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(text_ids)

def construct_input_ref_token_type_pair(input_ids, sep_ind=0):
    seq_len = input_ids.size(1)
    token_type_ids = torch.tensor([[0 if i <= sep_ind else 1 for i in range(seq_len)]], device=device)
    ref_token_type_ids = torch.zeros_like(token_type_ids, device=device)# * -1
    return token_type_ids, ref_token_type_ids

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids

def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

def custom_forward(inputs):
    preds = predict(inputs)
    return torch.softmax(preds, dim = 1)[0][1].unsqueeze(-1)

def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

In [ ]:
def dataframe_creation(attributions_sum, all_tokens, start, chrom):
  short_a_s = attributions_sum[0:len(all_tokens)]
  df = {'tokens':all_tokens, 'score':short_a_s}
  df = pd.DataFrame.from_dict(df)
  df = df[1:len(df)-1]
  starts = [start]
  ends = []
  l=list(df.tokens)
  for i in range(len(l))[1:]:
    starts.append(starts[i-1]+len(l[i-1]))
    ends.append(starts[i])
  ends.append(starts[-1]+len(l[-1]))
  df['chromStart'] = starts
  df['chromEnd']= ends
  df['chrom'] = [chrom]*len(df)
  return df

def bed_file_creation(df, filename):
  df_pos = df[df['score']>=0]
  df_neg = df[df['score']<0]
  mult = 1000/max(max(df_pos['score']), abs(min(df_neg['score'])))
  df_neg['score']= df_neg['score']*mult*(-1)
  df_pos['score']= df_pos['score']*mult
  pos_data = pd.DataFrame()
  pos_data['chrom'] = df_pos['chrom']
  pos_data['chromStart'] = df_pos['chromStart']
  pos_data['chromEnd'] = df_pos['chromEnd']
  pos_data['name'] = df_pos['tokens']
  pos_data['score']  = df_pos['score'].astype(int)
  pos = pybedtools.BedTool.from_dataframe(pos_data)
  pos.saveas(filename+'_pos.bed')
  neg_data = pd.DataFrame()
  neg_data['chrom'] = df_neg['chrom']
  neg_data['chromStart'] = df_neg['chromStart']
  neg_data['chromEnd'] = df_neg['chromEnd']
  neg_data['name'] = df_neg['tokens']
  neg_data['score']  = df_neg['score'].astype(int)
  neg = pybedtools.BedTool.from_dataframe(neg_data)
  neg.saveas(filename + '_neg.bed')

## Prepare and choose data

Reading data

In [ ]:
dataset_bed = pd.read_csv('/content/drive/MyDrive/SE_annotation_project/HEK293/data/SE_data.bed', sep='\t', comment='t', header=None)

In [ ]:
import Bio
from Bio import SeqIO

In [ ]:
path = '/content/drive/MyDrive/SE_annotation_project/HEK293/data/SE_data.fasta'

sequences = []
for seq_record in SeqIO.parse(path, "fasta"):
  if len(seq_record)< 25000:
    sequences.append(str(seq_record.seq.upper()))
  else:
    sequences.append(str(seq_record.seq.upper())[:25000])

In [ ]:
dataset = dataset_bed
dataset['text'] = sequences

In [ ]:
tokens = []
for i in dataset['text']:
  tokens.append(len(tokenizer.tokenize(i)))
dataset['tl'] = np.array(tokens)

In [ ]:
dataset = dataset[dataset['tl']<= 4096]

In [ ]:
probabilities = []
for text in dataset['text']:
  ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
  sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
  cls_token_id = tokenizer.cls_token_id

  lig = LayerIntegratedGradients(custom_forward, model.bert.embeddings)

  input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
  token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids, sep_id)
  position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
  attention_mask = construct_attention_mask(input_ids)
  indices = input_ids[0].detach().tolist()
  all_tokens = tokenizer.convert_ids_to_tokens(indices)

  probabilities.append(float(custom_forward(input_ids)[0]))

Attention type 'block_sparse' is not possible if sequence_length: 569 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


In [ ]:
dataset['probs'] = probabilities

<ipython-input-15-d23dc091a783>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['probs'] = probabilities


In [ ]:
top_SEs = dataset.sort_values(by = 'probs', ascending=False)[0:5]

In [ ]:
top_SEs

,0,1,2,3,4,text,tl,probs
306,chr17,28724096,28727062,SE_01_011200258,2966.0,TACTTTAATGGTTAAACAGTACCATAAGAGCATCACCCCCAATTTT...,436,0.967563
514,chr4,139555259,139558854,SE_01_011200471,3595.0,TTCTCGTTGTAAAATTATTTAATTAGAGGTTACAGCCTACTCTCAG...,584,0.966754
495,chr3,79016703,79019877,SE_01_011200528,3174.0,GGGAGTAGGAGACATACAGCATTAGATGTATCTTGTTAAAAACCTT...,483,0.966721
417,chr2,10119968,10123927,SE_01_011200295,3959.0,TGTAACTTTTGACTCATGGACTCCAAGTGCTTTGTTCATGCATTCT...,638,0.966699
550,chr5,126776113,126780010,SE_02_111400165,3897.0,AAATGTCCTATTCTTCCAGGATTGAAGGGATATTTTGTTAGAAGGC...,620,0.966454


We chose top-5 ranked by the model SEs

## Captum results for 5 top SEs

In [ ]:
score_viz = []
filenames = ['1', '2', '3', '4', '5']
for i in range(5):
  print(i)
  text = list(top_SEs['text'])[i]
  chrom = list(top_SEs[0])[i]
  start = list(top_SEs[1])[i]
  ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
  sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
  cls_token_id = tokenizer.cls_token_id

  lig = LayerIntegratedGradients(custom_forward, model.bert.embeddings)

  input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
  token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids, sep_id)
  position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
  attention_mask = construct_attention_mask(input_ids)
  indices = input_ids[0].detach().tolist()
  all_tokens = tokenizer.convert_ids_to_tokens(indices)

  attributions, delta = lig.attribute(inputs=input_ids,
                                    baselines=ref_input_ids,
                                    n_steps=700,
                                    internal_batch_size=1,
                                    return_convergence_delta=True)
  score = predict(input_ids)
  attributions_sum = summarize_attributions(attributions)
  score_viz.append(viz.VisualizationDataRecord(attributions_sum,
                                        torch.softmax(score, dim = 1)[0][1],
                                        torch.argmax(torch.softmax(score, dim = 0)[0]),
                                        1,
                                        text,
                                        attributions_sum.sum(),
                                        all_tokens,
                                        delta))
  df = dataframe_creation(attributions_sum.cpu(), all_tokens, start, chrom)
  name = '/content/drive/MyDrive/SE_annotation_project/HEK293/score_files/' + filenames[i]
  bed_file_creation(df, name)

0


<ipython-input-7-4bb86bc49fa3>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_neg['score']= df_neg['score']*mult*(-1)
<ipython-input-7-4bb86bc49fa3>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pos['score']= df_pos['score']*mult


1


<ipython-input-7-4bb86bc49fa3>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_neg['score']= df_neg['score']*mult*(-1)
<ipython-input-7-4bb86bc49fa3>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pos['score']= df_pos['score']*mult


2


<ipython-input-7-4bb86bc49fa3>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_neg['score']= df_neg['score']*mult*(-1)
<ipython-input-7-4bb86bc49fa3>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pos['score']= df_pos['score']*mult


3


<ipython-input-7-4bb86bc49fa3>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_neg['score']= df_neg['score']*mult*(-1)
<ipython-input-7-4bb86bc49fa3>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pos['score']= df_pos['score']*mult


4


<ipython-input-7-4bb86bc49fa3>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_neg['score']= df_neg['score']*mult*(-1)
<ipython-input-7-4bb86bc49fa3>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pos['score']= df_pos['score']*mult


In [ ]:
print('\033[1m', 'Visualization For Score', '\033[0m')
viz.visualize_text([score_viz[0]])

 Visualization For Score 


In [ ]:
print('\033[1m', 'Visualization For Score', '\033[0m')
viz.visualize_text([score_viz[1]])

 Visualization For Score 


In [ ]:
print('\033[1m', 'Visualization For Score', '\033[0m')
viz.visualize_text([score_viz[2]])

 Visualization For Score 


In [ ]:
print('\033[1m', 'Visualization For Score', '\033[0m')
viz.visualize_text([score_viz[3]])

 Visualization For Score 


In [ ]:
print('\033[1m', 'Visualization For Score', '\033[0m')
viz.visualize_text([score_viz[4]])

 Visualization For Score 
